<img src="fcn_model.png">

Reference 1 : https://github.com/shekkizh/FCN.tensorflow/blob/master/FCN.py
<br>Reference 2 : https://modulabs-biomedical.github.io/FCN

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets

C:\Users\user\Anaconda3\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# image size

height = 224
width = 224
num_of_channels = 3
num_of_classes = 21

keep_prob = 0.5

In [3]:
image = tf.placeholder(tf.float32, [None, height, width, num_of_channels])

In [6]:
def fcn8_with_pretrained_vgg16(image, num_of_classes, keep_prob):
    "Extracting Layer from Pre-trained VGG16"
    vgg16 = nets.vgg.vgg_16(image, num_classes = num_of_classes, dropout_keep_prob = keep_prob)
    
    pool3 = tf.get_default_graph().get_tensor_by_name('vgg_16/pool3/MaxPool:0')
    pool4 = tf.get_default_graph().get_tensor_by_name('vgg_16/pool4/MaxPool:0')
    pool5 = tf.get_default_graph().get_tensor_by_name('vgg_16/pool5/MaxPool:0')
    
    "Feature-level Classificaiton"
    # output shape = [None, H/32, H/32, 4096]
    conv6 = slim.conv2d(pool5, 4096, [7, 7], scope = 'conv6') # (7*7*256 + 1) * 4,096 = 51,384,320
    conv6 = tf.nn.dropout(conv6, keep_prob = keep_prob)
    
    # output shape = [None, H/32, H/32, 4096]
    conv7 = slim.conv2d(conv6, 4096, [1, 1], scope = 'conv7') # (1*1*4,096 + 1) * 4,096 = 16,781,312
    conv7 = tf.nn.dropout(conv7, keep_prob = keep_prob)
    
    # output shape = [None, H/32, H/32, num_of_classes]
    conv8 = slim.conv2d(conv7, num_of_classes, [1, 1], scope = 'conv8') # (1*1*4,096 + 1) * 21(num_of_classes) = 86,037
    
    "Upsampling"
    conv_t1 = slim.conv2d_transpose(conv8, num_outputs = pool4.get_shape()[3], kernel_size = [4, 4], stride = 2)
    # (4*4*21 + 1) * 384 = 129,408
    fuse_1 = tf.add(conv_t1, pool4, name = 'fuse_1')
    
    conv_t2 = slim.conv2d_transpose(fuse_1, num_outputs = pool3.get_shape()[3], kernel_size = 4, stride = 2) 
    # (4*4*384 + 1) * 384 = 2,359,680
    fuse_2 = tf.add(conv_t2, pool3, name = 'fuse_2')
    
    conv_t3 = slim.conv2d_transpose(fuse_2, num_outputs = num_of_channels, kernel_size = 16, stride = 8)
    # (16*16*384 + 1) * 3 = 294,915
    
    "Segmentation"
    annotation_pred = tf.argmax(conv_t3, dimension = num_of_channels, name="prediction")
    
    return tf.expand_dims(annotation_pred, dim = num_of_channels)

In [7]:
fcn8_with_pretrained_vgg16 = fcn8_with_pretrained_vgg16(image, num_of_classes, keep_prob)

In [18]:
print("The Output Shape of FCN8 : " + str(fcn8_with_pretrained_vgg16.shape))

The Output Shape of FCN8 : (?, 224, 224, 1)
